In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.avro.functions import from_avro, to_avro
import pandas as pd
import json

In [2]:
#Spark Session creation configured to interact with Kafka
spark = SparkSession.builder.appName("pyspark-notebook").\
config("spark.jars.packages","org.apache.spark:spark-sql-kafka-0-10_2.12:3.0.0,org.apache.spark:spark-avro_2.12:3.0.0,org.mongodb.spark:mongo-spark-connector_2.12:3.0.0").\
getOrCreate()

22/10/08 14:49:05 WARN Utils: Your hostname, Daos-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.182.169 instead (on interface en0)
22/10/08 14:49:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/opt/homebrew/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /Users/daoanhtuan/.ivy2/cache
The jars for the packages stored in: /Users/daoanhtuan/.ivy2/jars
org.apache.spark#spark-sql-kafka-0-10_2.12 added as a dependency
org.apache.spark#spark-avro_2.12 added as a dependency
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-bfd73497-a907-4c22-98f1-f72161bcde60;1.0
	confs: [default]
	found org.apache.spark#spark-sql-kafka-0-10_2.12;3.0.0 in central
	found org.apache.spark#spark-token-provider-kafka-0-10_2.12;3.0.0 in central
	found org.apache.kafka#kafka-clients;2.4.1 in central
	found com.github.luben#zstd-jni;1.4.4-3 in central
	found org.lz4#lz4-java;1.7.1 in central
	found org.xerial.snappy#snappy-java;1.1.7.5 in central
	found org.slf4j#slf4j-api;1.7.30 in central
	found org.spark-project.spark#unused;1.0.0 in central
	found org.apache.commons#commons-pool2;2.6.2 in central
	found org.apache.spark#spark-avro_2.12;3.0.0 in central
	fo

22/10/08 14:49:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [8]:
#Read data from Kafka
data = spark\
  .readStream\
  .format("kafka")\
  .option("kafka.bootstrap.servers", "ec2-35-180-61-109.eu-west-3.compute.amazonaws.com:9092")\
  .option("subscribe", "twitter_demo")\
  .option("startingOffsets", "earliest")\
  .load()\
  .selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)").alias("data").select("data.value")

In [ ]:
#write streaming data as a text file
data.\
writeStream.\
format("text").\
option("checkpointLocation", "checkpoint/schema").\
option("format", "text").\
option("path", "schema/in").\
outputMode("append").\
start()

In [ ]:
#extract schema by reading the file written above
smallBatchSchema = spark.read.json("schema/in/*.txt").schema

In [ ]:
#write schema as JSON file to schema folder
with open("schema/out/tweet_schema.json", "w") as f:
    json.dump(smallBatchSchema.jsonValue(), f)